In [82]:
!pip install geopy

In [83]:
!pip install bs4  --upgrade

Requirement already up-to-date: bs4 in /home/mark/anaconda3/lib/python3.6/site-packages (0.0.1)


In [84]:
!pip install beautifulsoup4

In [85]:
from requests import get
# from requests.exceptions import RequestExceptionA
from contextlib import closing
from bs4 import BeautifulSoup
#import beautifulsoup4
#from Beautifulsoup4 import BeautifulSoup
import bs4
import re
import regex
import pandas as pd
from geopy.distance import distance
import datetime
now = datetime.datetime.now()

In [86]:
bs4.__version__

'4.6.3'

In [87]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [88]:
#url = "https://ftw.usatoday.com/2016/08/ranking-best-worst-mlb-stadiums-30-petco-wrigley-pnc"
url = "https://en.wikipedia.org/wiki/List_of_current_Major_League_Baseball_stadiums"
raw_html = simple_get(url)
html = BeautifulSoup(raw_html, 'html.parser')

table = html.select('table')[1]
teams = []
stadium = None
for tr in table.select('tr'):
    if tr.has_attr('style'):
        # print('tr style', tr['style'])
        tds = [td for td in tr.select('td')]
        if len(tds) > 6:
            teams.append((tds[5].a.text, tds[1].a.text))
    else :
        stadium = tr.find_next('th', {'scope': 'row'}).a.text
        tds = [td for td in tr.select('td')]
        if len(tds) > 5:
            team = tds[4].a.text
            teams.append((team, stadium))
            if team == 'New York Yankees':
                break 


print(len(teams))
#print(teams)
# for team in teams:
#     print(team)

30


In [89]:
gurl = "https://en.wikipedia.org/wiki/"
p = regex.compile(r"(\-?[\d\.]+)\;\s+(\-?[\d\.]+)")
lat = 0
long = 0
rows = []
# teams = [('Arizona Diamondbacks','Chase Field' )]
# teams = sorted(teams, key = lambda team: team[1])
for team, stadium in teams:
    # print(team, stadium)
    raw_html = simple_get(gurl  + stadium.replace(' ', '_')).decode("utf-8") 
    html = BeautifulSoup(raw_html, 'html.parser')
    geo = html.select('span .geo')
    # print(f"geo: {geo}")
    m = regex.match(p, geo[0].text)
    rows.append({'team': team, 'stadium': stadium, 'lat': float(m.group(1)), 'long': float(m.group(2))})
teamd = pd.DataFrame(rows)        
# teamd

In [93]:
def print_distance(winner, loser, rows):
    w = teamd[teamd['team'] == winner]
    l = teamd[teamd['team'] == loser]
    if len(w.lat.values) == 0:
        print('unknown winning team', winner)
    if len(l.lat.values) == 0:
        print('unknown losing team', loser)                  
    d = distance((w.lat.values[0], w.long.values[0]),
                 (l.lat.values[0], l.long.values[0])).miles
    rows.append({'year': year, 'winner': winner, 'loser': loser, 'distance': d})

 [after 1957 season mlb moves west](https://en.wikipedia.org/wiki/Major_League_Baseball_relocation_of_1950s%E2%80%9360s)

In [94]:
wsurl = "https://en.wikipedia.org/wiki/List_of_World_Series_champions"
raw_html = simple_get(wsurl)
html = BeautifulSoup(raw_html, 'html.parser')
win = True
winner = None
rows = []
year = datetime.datetime.now().year
for tr in html.select('tr'):
    th = tr.find_next('th', attrs={'scope':'row'})
    a = th.a
    series_year = int(a.text)
#     if series_year == year:
         # necessary iff world series hasn't been decided yet this year (picked my team to win)
         # edit in the teams
#         print_distance('Boston Red Sox', 'Los Angeles Dodgers', rows)
#         break
    if series_year > 1958: # before 1958 no west coast teams so no maximum distance       
        for td in tr.select('td'):
            if (td.has_attr('style')):
                tda = td.a
                if win:
                    winner = tda.text
                    if winner == 'Florida Marlins':  # convert old name to current name
                        winner = 'Miami Marlins'
                    elif winner == 'Anaheim Angels': # convert old name to current name
                        winner = 'Los Angeles Angels'
                else:
                    print_distance(winner, tda.text, rows)
                win = False if win else True
    if series_year == year:
        # necessary iff world series has been decided this year
        break
# print(len(rows))
distances = pd.DataFrame(rows)
# distances

In [95]:
distances.loc[distances['distance'].idxmax()]

distance                2593.77
loser       Los Angeles Dodgers
winner           Boston Red Sox
year                       2018
Name: 58, dtype: object